In [2]:
from datetime import datetime ,timedelta
import warnings
warnings.filterwarnings('ignore')


# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd 
from sqlalchemy import create_engine, select
import sqlite3
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet','stopwords','omw-1.4'])

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB , BernoulliNB
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay , classification_report , recall_score , precision_score , f1_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///Database.db')
df =  pd.read_sql(
    "SELECT * FROM messagesdf",
    con=engine )
X = df[['id','message' ,'original' ,'genre']]
Y = df.drop(['id','message' ,'original' ,'genre'], axis =1 )

In [6]:
Y.drop('child_alone' , axis = 1 , inplace=True)

### 2. Write a tokenization function to process your text data

In [7]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
url_place_holder_string="urlplaceholder"
swords = stopwords.words('english')
def tokenize(text):


    
    # Replace url with a url placeholder string
    detected_urls = re.findall(url_regex, text)
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)
        

    lista = ["[^a-zA-Z]" , "http\S+" ,"http" ,"@\S+" ,  "[^A-Za-z0-9(),!?@\'\`\"\_\n]" , "@" ]
    for item in lista:
        text = re.sub(item, " ", text)



    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    # Remove stopwords
    tokens = [tok.lower().strip() for tok in tokens if tok.lower() not in swords]

    # List of cleaned tokens
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens ]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
vect = CountVectorizer(tokenizer=tokenize, min_df=50, analyzer='word')
tfidf = TfidfTransformer(smooth_idf=True)
clf =MultinomialNB()

In [9]:
test = vect.fit_transform(X['message'])

In [11]:
print(f'Número de palavras: {len(vect.vocabulary_)}')

Número de palavras: 1312


In [13]:
estimators = [('vect' , CountVectorizer(tokenizer=tokenize, min_df=50)) , \
              ('tfidf' ,TfidfTransformer() ) , \
              ('clf',MultiOutputClassifier(AdaBoostClassifier()))]
pipeline = Pipeline(estimators)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
x_train, x_test, y_train , y_test = train_test_split(X, Y , test_size=0.2, random_state=1)

In [15]:
pipeline.fit(x_train['message'], y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(min_df=50,
                                 tokenizer=<function tokenize at 0x000001F2D9A805E0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [16]:
pipeline.score(x_train['message'], y_train)

0.25510886172650876

In [17]:
pipeline.score(x_test['message'], y_test)

0.23376623376623376

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.